In [1]:
%reload_ext lab_black

In [70]:
import numpy as np
from scipy.optimize import minimize_scalar
from worstcase import param, derive, unit
from pprint import pprint

In [3]:
def ebers_moll_model(Is, Vbe, T):
    q = 1.602176e-19 * unit.C
    k = 1.3806503e-23 * unit("J/K")
    return Is * (np.exp(q * Vbe / (k * T)) - 1)

In [4]:
def dmmt3906(Vbe, T):
    return ebers_moll_model(20.3 * unit.fA, Vbe, T).to(unit.mA)

In [5]:
def dmmt3906_unitless(Vbe, T):
    return dmmt3906(Vbe * unit.V, T * unit.K).to(unit.A).m

In [138]:
VBE_MISMATCH = param.bytol(nom=0 * unit.mV, tol=2 * unit.mV, rel=False)
HFE_MISMATCH = param.bytol(nom=0, tol=0.02, rel=False)
HFE = param.byrange(nom=150, lb=80, ub=300)
R1 = param.bytol(nom=3.6 * unit.kohm, tol=0.01, rel=True)
R2 = param.bytol(nom=3.3 * unit.kohm, tol=0.01, rel=True)

In [139]:
@derive.byev(
    r1=R1.nom, r2=R2.nom, hfe1=HFE, vbe_mismatch=VBE_MISMATCH, hfe_mismatch=HFE_MISMATCH
)
def IDIODE(vout, vgsth, rdson, r1, r2, hfe1, vbe_mismatch, hfe_mismatch, temp):
    # the system of equations will be solved numerically
    def idiode_error(idiode_estimate):
        def inverse_dmmt3906(ic):
            vbefun = lambda vbe: dmmt3906_unitless(vbe, temp.to(unit.K).m)
            minfun = lambda vbe: (vbefun(vbe) - ic.to(unit.A).m) ** 2
            result = minimize_scalar(minfun)
            assert result.success
            return result.x * unit.V

        vc2 = vout - vgsth  # pass transistor is "barely on" (at threshold)
        ic2 = vc2 / r2
        vbe2 = inverse_dmmt3906(ic2)
        hfe2 = hfe1 * (1 + hfe_mismatch)
        ib2 = ic2 / hfe2
        vb = vout - vbe2

        ie1 = vb / r1 - ib2
        ic1 = ie1 * hfe1 / (1 + hfe1)
        ib1 = ic1 / hfe1
        vbe1 = inverse_dmmt3906(ic1)
        vbe1 += vbe_mismatch
        vin = vb + vbe1

        # print("Q2:", vout, vb, vc2, ic2.to(unit.mA), ib2.to(unit.mA))
        # print("Q1:", vin, ic1.to(unit.mA), ib1.to(unit.mA))

        idiode_actual = (vin - vout) / rdson
        return (idiode_actual.to(unit.A).m - idiode_estimate) ** 2

    result = minimize_scalar(idiode_error)
    assert result.success
    return result.x * unit.A

In [140]:
RDSON = param.bytol(nom=600 * unit.mohm, tol=200 * unit.mohm, rel=False)
VGSTH = param.bytol(nom=1.0 * unit.V, tol=200 * unit.mV, rel=False)

In [141]:
idiode = IDIODE(
    vout=3.3 * unit.V,
    vgsth=VGSTH,
    rdson=RDSON,
    temp=273 * unit.K,
)  # note: SI2301CDS mosfet with ~330mohm series resistance

In [142]:
idiode_ev = idiode(tag="extreme-value")()

In [143]:
idiode_pe = idiode(tag="pass-element").ss([VGSTH, RDSON])

In [144]:
idiode_mp = idiode(tag="matched-pair").ss([VBE_MISMATCH, HFE_MISMATCH, HFE])

In [145]:
pprint([idiode_ev, idiode_pe, idiode_mp])

[extreme-value: 2.797 mA (nom), -6.484 mA (lb), 14.99 mA (ub),
 pass-element: 2.797 mA (nom), -783 µA (lb), 9.623 mA (ub),
 matched-pair: 2.797 mA (nom), -984.6 µA (lb), 6.383 mA (ub)]


In [146]:
uc_pe = (idiode_pe.ub - idiode_pe.lb) / 2  # pass-element uncertainty contrib
uc_mp = (idiode_mp.ub - idiode_mp.lb) / 2  # matched-pair uncertainty contrib
uc_br = 1 * unit.mA  # bias-resistors uncertainty contrib

In [147]:
np.sqrt(uc_pe ** 2 + uc_mp ** 2 + uc_br ** 2).to(unit.mA)  # RSS uncertainty

6.453205740710734 <Unit('milliampere')>

In [117]:
max_current = 125 * unit.mA
(RDSON.ub * max_current).to(unit.mV)  # max forward voltage drop when USB unpowered

100.0 <Unit('millivolt')>